In [ ]:
# %pip install -U -q pydantic
from pydantic import BaseModel, Field
import json
import re
# %pip install -U -q pandas
import pandas as pd
# %pip install -U -q langchain_ollama
from langchain_ollama import ChatOllama
# %pip install -U -q langchain-openai
from langchain_openai import ChatOpenAI
# %pip install -U -q neo4j
from neo4j import GraphDatabase
#%pip install -U -q spacy
#import spacy

llm = ChatOllama(model="llama3.2", temperature=0, max_tokens=2048) 